In [1]:

import tensorflow.keras as keras
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import uproot
import pandas as pd
import awkward as ak
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import tensorflow_decision_forests as tfdf
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots
from keras.callbacks import LearningRateScheduler


2022-10-18 14:15:19.506012: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-18 14:15:21.043210: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/benk/root/root-6.26.06-install/lib
2022-10-18 14:15:21.043305: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-18 14:15:21.295215: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-1

In [2]:

def open_root_files(file_names,tree):
    file = uproot.open(file_names)
    tree_name = file[tree]
    return tree_name

def create_heat_map(df):
    corr = df.corr()
    sns.heatmap(corr, 
    cmap='RdYlGn', 
    xticklabels=corr.columns.values,
    yticklabels=corr.columns.values)
    plt.show()

def create_tensor_object(train_variables,dict):
    df = pd.DataFrame()
    key = list(dict.keys())[0]
    for var in train_variables:
        if var == "classification":
            continue
        else:
            #df[var] = np.array(dict[key][var].array())
            df.insert(len(df.columns), var, dict[key][var].array())
            print(dict[key][var].array())
    if "signal" in key:
        print("SIGNAL CLASSIFICATION SET TO 1", key)
        df.insert(0, 'classification', 1)
    else:
        print("BACKGROUND CLASSIFICATION SET TO 0", key)
        df.insert(0, 'classification', 0)
    
    #split the data into train and testing set
    labels = df.iloc[:,0]
    features = df.iloc[:, 1:]
    
    train_df_1, test_df, labels_train_1, labels_test = train_test_split(features, labels, test_size=0.2)
    
    train_df, val_df, labels_train, labels_val = train_test_split(train_df_1, labels_train_1, test_size=0.1)
   
    # numerical_features = features.select_dtypes(include=['float64', 'int64', 'float32', 'int32'])
    # numerical_columns = numerical_features.columns
    # print(type(numerical_columns))
    ct = ColumnTransformer([("only numeric", StandardScaler(), [0,12])], remainder = 'passthrough')
    features_train_scaled = ct.fit_transform(train_df)
    features_val_scaled = ct.transform(val_df)
    features_test_scaled = ct.transform(test_df) 
    print(features_train_scaled.shape)   
    
    #create heat map of training variables
    hmap = create_heat_map(train_df)
    
    # return features_train_scaled, features_test_scaled, labels_train, labels_test, features_val_scaled, labels_val
    return train_df, test_df, labels_train, labels_test, val_df, labels_val
    #return 0, 0, 0, 0, 0, 0

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.show()
    
def plot_loss(fit):
    plt.plot(fit.history['loss'])
    plt.plot(fit.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

def plot_accuracy(fit):
    plt.plot(fit.history['accuracy'])
    plt.plot(fit.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
def custom_LearningRate_schedular(epoch):
    if epoch < 5:
        return 0.01
    else:
        return 0.01 * tf.math.exp(0.1 * (10 - epoch))

    
def get_model(train_tensor):
    model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(len(train_tensor[0])-1,)),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.2),
    # keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    # keras.layers.Dense(4, activation='relu'),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
    ])
    model.compile(optimizer=tf.optimizers.SGD(learning_rate=0.0001),
                loss=tf.keras.losses.BinaryCrossentropy(
                        name='binary_crossentropy'),
                metrics=['accuracy', 
                        keras.metrics.AUC(name='auc'),
                        keras.metrics.AUC(name='prc', curve='PR')])
    return model

def boosted_decision_tree():
    model = tfdf.keras.GradientBoostedTreesModel()
    return model

In [3]:
#open the files
signal_file_e = open_root_files("mc16e_signal.root","nominal")
#signal_file_d = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16d_signal.root","nominal")
#signal_file_a = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16a_signal.root","nominal")
ttbar_file_e = open_root_files("mc16e_ttbar.root","nominal")
#ttbar_file_d = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16d_ttbar.root","nominal")
#ttbar_file_a = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16a_ttbar.root","nominal")
#wjets_file_e = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16e_wjets.root","nominal")
#wjets_file_d = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16d_wjets.root","nominal")
#wjets_file_a = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16a_wjets.root","nominal")
#diboson_file_e = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16e_diboson.root","nominal")
#diboson_file_d = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16d_diboson.root","nominal")
#diboson_file_a = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16a_diboson.root","nominal")
#zjets_file_e = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16e_zjets.root","nominal")
#zjets_file_d = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16d_zjets.root","nominal")
#zjets_file_a = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16a_zjets.root","nominal")
#singletop_file_e = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16e_singletop.root","nominal")
#singletop_file_d = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16d_singletop.root","nominal")
#singletop_file_a = open_root_files("~/Physics/HWWAnalysis/ntuples/mc16a_singletop.root","nominal")

In [4]:
train_variables = ['classification', 'mupt_cand', 'mueta_cand', 'muphi_cand', 'ljet_pt_cand', 'ljet_eta_cand',\
                   'ljet_phi_cand', 'ljet_mass_cand', 'dR_values_cand', 'pt_higgs',\
                   'mass_T', 'met_met', 'met_phi', 'mass_mj', 'weight']

#make the training and testing samples for each period
signal_train_e,  signal_test_e, signal_train_out_e, signal_test_out_e, signal_val_e, signal_val_out_e = create_tensor_object(train_variables, {"signal_e" : signal_file_e})
# signal_train_d,  signal_test_d, signal_train_out_d, signal_test_out_d, signal_val_d, signal_val_out_d = create_tensor_object(train_variables , {"signal_d" : signal_file_d})
# signal_train_a, signal_test_a, signal_train_out_a, signal_test_out_a, signal_val_a, signal_val_out_a = create_tensor_object(train_variables , {"signal_a" : signal_file_a})
ttbar_train_e,  ttbar_test_e, ttbar_train_out_e, ttbar_test_out_e, ttbar_val_e, ttbar_val_out_e = create_tensor_object(train_variables , {"ttbar_e" : ttbar_file_e})
# ttbar_train_d,  ttbar_test_d, ttbar_train_out_d, ttbar_test_out_d, ttbar_val_d, ttbar_val_out_d = create_tensor_object(train_variables , {"ttbar_d" : ttbar_file_d})
# ttbar_train_a,  ttbar_test_a, ttbar_train_out_a, ttbar_test_out_a, ttbar_val_a, ttbar_val_out_a = create_tensor_object(train_variables , {"ttbar_a" : ttbar_file_a})
# wjets_train_e,  wjets_test_e, wjets_train_out_e, wjets_test_out_e, wjets_val_e, wjets_val_out_e = create_tensor_object(train_variables , {"wjets_e" : wjets_file_e})
# wjets_train_d,  wjets_test_d, wjets_train_out_d, wjets_test_out_d, wjets_val_d, wjets_val_out_d = create_tensor_object(train_variables , {"wjets_d" : wjets_file_d})
# wjets_train_a,  wjets_test_a, wjets_train_out_a, wjets_test_out_a, wjets_val_a, wjets_val_out_a = create_tensor_object(train_variables , {"wjets_a" : wjets_file_a})
# diboson_train_e,  diboson_test_e, diboson_train_out_e, diboson_test_out_e, diboson_val_e, diboson_val_out_e = create_tensor_object(train_variables , {"diboson_e" : diboson_file_e})
# diboson_train_d,  diboson_test_d, diboson_train_out_d, diboson_test_out_d, diboson_val_d, diboson_val_out_d = create_tensor_object(train_variables , {"diboson_d" : diboson_file_d})
# diboson_train_a,  diboson_test_a, diboson_train_out_a, diboson_test_out_a, diboson_val_a, diboson_val_out_a = create_tensor_object(train_variables , {"diboson_a" : diboson_file_a})
# zjets_train_e,  zjets_test_e, zjets_train_out_e, zjets_test_out_e, zjets_val_e, zjets_val_out_e = create_tensor_object(train_variables , {"zjets_e" : zjets_file_e})
# zjets_train_d,  zjets_test_d, zjets_train_out_d, zjets_test_out_d, zjets_val_d, zjets_val_out_d = create_tensor_object(train_variables , {"zjets_d" : zjets_file_d})
# zjets_train_a,  zjets_test_a, zjets_train_out_a, zjets_test_out_a, zjets_val_a, zjets_val_out_a = create_tensor_object(train_variables , {"zjets_a" : zjets_file_a})
# singletop_train_e, singletop_test_e, singletop_train_out_e, singletop_test_out_e, singletop_val_e, singletop_val_out_e = create_tensor_object(train_variables , {"singletop_e" : singletop_file_e})
# singletop_train_d,singletop_test_d, singletop_train_out_d, singletop_test_out_d, singletop_val_d, singletop_val_out_d = create_tensor_object(train_variables , {"singletop_d" : singletop_file_d})
# singletop_train_a, singletop_test_a, singletop_train_out_a, singletop_test_out_a, singletop_val_a, singletop_val_out_a = create_tensor_object(train_variables , {"singletop_a" : singletop_file_a})


# train_dataset = tf.concat([signal_train_e, signal_train_d, signal_train_a, ttbar_train_e,\
#                             ttbar_train_d, ttbar_train_a, wjets_train_e, wjets_train_d,\
#                             wjets_train_a, diboson_train_e, diboson_train_d, diboson_train_a,\
#                             zjets_train_e, zjets_train_d, zjets_train_a, singletop_train_e,\
#                             singletop_train_d, singletop_train_a], axis=0)


# test_dataset = tf.concat([signal_test_e, signal_test_d, signal_test_a, ttbar_test_e,\
#                             ttbar_test_d, ttbar_test_a, wjets_test_e, wjets_test_d,\
#                             wjets_test_a, diboson_test_e, diboson_test_d, diboson_test_a,\
#                             zjets_test_e, zjets_test_d, zjets_test_a, singletop_test_e,\
#                             singletop_test_d, singletop_test_a], axis=0)

# val_dataset = tf.concat([signal_val_e, signal_val_d, signal_val_a, ttbar_val_e,\
#                             ttbar_val_d, ttbar_val_a, wjets_val_e, wjets_val_d,\
#                             wjets_val_a, diboson_val_e, diboson_val_d, diboson_val_a,\
#                             zjets_val_e, zjets_val_d, zjets_val_a, singletop_val_e,\
#                             singletop_val_d, singletop_val_a], axis=0)

# train_output = tf.concat([signal_train_out_e, signal_train_out_d, signal_train_out_a, ttbar_train_out_e,\
#                             ttbar_train_out_d, ttbar_train_out_a, wjets_train_out_e, wjets_train_out_d,\
#                             wjets_train_out_a, diboson_train_out_e, diboson_train_out_d, diboson_train_out_a,\
#                             zjets_train_out_e, zjets_train_out_d, zjets_train_out_a, singletop_train_out_e,\
#                             singletop_train_out_d, singletop_train_out_a], axis=0)

# test_output = tf.concat([signal_test_out_e, signal_test_out_d, signal_test_out_a, ttbar_test_out_e,\
#                             ttbar_test_out_d, ttbar_test_out_a, wjets_test_out_e, wjets_test_out_d,\
#                             wjets_test_out_a, diboson_test_out_e, diboson_test_out_d, diboson_test_out_a,\
#                             zjets_test_out_e, zjets_test_out_d, zjets_test_out_a, singletop_test_out_e,\
#                             singletop_test_out_d, singletop_test_out_a], axis=0)

# val_output = tf.concat([signal_val_out_e, signal_val_out_d, signal_val_out_a, ttbar_val_out_e,\
#                             ttbar_val_out_d, ttbar_val_out_a, wjets_val_out_e, wjets_val_out_d,\
#                             wjets_val_out_a, diboson_val_out_e, diboson_val_out_d, diboson_val_out_a,\
#                             zjets_val_out_e, zjets_val_out_d, zjets_val_out_a, singletop_val_out_e,\
#                             singletop_val_out_d, singletop_val_out_a], axis=0)

train_dataset = tf.concat([signal_train_e, ttbar_train_e], axis=0)
test_dataset = tf.concat([signal_test_e, ttbar_test_e], axis=0)
val_dataset = tf.concat([signal_val_e, ttbar_val_e], axis=0)
train_output = tf.concat([signal_train_out_e, ttbar_train_out_e], axis=0)
val_output = tf.concat([signal_val_out_e, ttbar_val_out_e], axis=0)
test_output = tf.concat([signal_test_out_e, ttbar_test_out_e], axis=0)

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [ ]:
nn_model = get_model(train_dataset)
# #fit the model to train on all but the last column
print("MATT, FITTING MODEL")
callback = LearningRateScheduler(custom_LearningRate_schedular)
# print(train_dataset[:,train_dataset.shape[1]-1 : train_dataset.shape[1]])
nn_fit = nn_model.fit(train_dataset[:, 0:train_dataset.shape[1]-1], train_output, epochs=500, batch_size = 500, validation_data=(val_dataset[:, 0:train_dataset.shape[1]-1], val_output), sample_weight=train_dataset[:,train_dataset.shape[1]-1 : train_dataset.shape[1]], shuffle=True)
# validation_data=(val_dataset[:, 0:train_dataset.shape[1]-1], val_output),
# print(train_dataset[:,0:train_dataset.shape[1]-1])
# nn_fit = nn_model.fit(train_dataset[:,0:train_dataset.shape[1]-1], train_output[:,0:0:train_dataset.shape[1]-1], epochs=70, batch_size=500, verbose=1, shuffle=True, validation_data=(val_dataset[:,0:train_dataset.shape[1]-1], val_output[:,0:train_dataset.shape[1]-1]), sample_weight=train_dataset[:,train_dataset.shape[1]-1:train_dataset.shape[1]])
print("MATT, MODEL FITTED")
print("MATT, PREDICTING")
y_scores = nn_model.predict(test_dataset[:, 0:train_dataset.shape[1]-1])



bdt_model = boosted_decision_tree()
print("MATT, FITTING MODEL")
bdt_fit = bdt_model.fit(train_dataset[:, 0:train_dataset.shape[1]-1], train_output, sample_weight=train_dataset[:,train_dataset.shape[1]-1 : train_dataset.shape[1]])
print("MATT, MODEL FITTED")
print("MATT, PREDICTING")
bdt_y_scores = bdt_model.predict(test_dataset[:, 0:train_dataset.shape[1]-1])


NameError: name 'train_dataset' is not defined

In [ ]:
# model.summary()
# model.make_inspector().features()
# print(bdt_model.summary())
print(bdt_y_scores)
nn_signal_scores = y_scores[test_output == 1]
nn_background_scores = y_scores[test_output == 0]

bdt_signal_scores = bdt_y_scores[test_output == 1]
bdt_background_scores = bdt_y_scores[test_output == 0]

nn_fakes, nn_reals, thresholds = roc_curve(test_output, y_scores)
bdt_fakes, bdt_reals, bdt_thresholds = roc_curve(test_output, bdt_y_scores)

print("NN AUC: ", auc(nn_fakes, nn_reals))
print("BDT_AUC: ", auc(bdt_fakes, bdt_reals))

nn_loss_plot = plot_loss(nn_fit)
nn_accuracy_plot = plot_accuracy(nn_fit)
# bdt_loss_plot = plot_loss(bdt_fit)
# bdt_accuracy_plot = plot_accuracy(bdt_fit)
# plotter = tfdocs.plots.HistoryPlotter(metric = 'binary_crossentropy', smoothing_std=10)
# plotter.plot(nn_fit)

plot_roc_curve(nn_fakes, nn_reals)
plot_roc_curve(bdt_fakes, bdt_reals)

#plot signal and background scores
plt.plot(bdt_fakes, bdt_reals, label="BDT")
plt.plot(nn_fakes, nn_reals, label="NN")
plt.legend()
plt.show()

plt.hist(nn_signal_scores, bins=100, range=(0,1), alpha = 0.5, label='signal')
plt.hist(nn_background_scores, bins=100, range=(0,1), alpha = 0.5, label='background')
plt.xlabel('NN score')
plt.ylabel('a.u.')
# plt.yscale('log')
plt.show()
plt.hist(nn_background_scores, bins=100, range=(0,1), alpha = 0.5, label='background')
# plt.yscale('log')
plt.show()

plt.hist(bdt_signal_scores, bins=100, range=(0,1), alpha = 0.5, label='signal')
plt.xlabel('BDT score')
plt.ylabel('a.u.')
plt.yscale('log')
plt.show()
plt.hist(bdt_background_scores, bins=100, range=(0,1), alpha = 0.5, label='background')
plt.yscale('log')
plt.show()
